# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
#Your code here; import the necessary packages

In [2]:
import sqlite3
import pandas as pd

In [3]:
#Your code here; create the connection and cursor

In [4]:
conn = sqlite3.Connection('data.sqlite')
c = conn.cursor()

## Select the Average Number of Orders on a Product by Product Basis

In [5]:
#Your code here; use a subquery. No join will be necessary.

In [6]:
df = pd.read_sql_query('''SELECT AVG(nOrders)
                            FROM (SELECT productCode,
                                 COUNT(orderNumber) as nOrders
                                 FROM orderdetails 
                                 GROUP BY 1);''', conn)
df.head(10)


,AVG(nOrders)
0,27.486239


## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [7]:
#Your code here; use a subquery. No join will be necessary.

In [8]:
df = pd.read_sql_query('''SELECT customerNumber, contactLastName, contactFirstName
                            FROM customers 
                            WHERE customerNumber in (SELECT customerNumber 
                                                        FROM orders 
                                                        WHERE orderDate = '2003-01-31');''', conn)
df.head()
  

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [9]:
#Your code here

In [11]:
df = pd.read_sql_query('''SELECT productName, count(orderNumber)
                            FROM products
                            JOIN orderdetails USING (productCode)
                            GROUP BY 1
                            ORDER by count(orderNumber);''', conn)
df.head()

,productName,count(orderNumber)
0,1952 Citroen-15CV,24
1,1957 Ford Thunderbird,24
2,1911 Ford Town Car,25
3,1917 Grand Touring Sedan,25
4,1932 Alfa Romeo 8C2300 Spider Sport,25


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [13]:
#Your code here
df = pd.read_sql_query("""select productName,
                    count(distinct customerNumber) numPurchasers
                    from products
                    join orderdetails
                    using(productCode)
                    join orders
                    using (orderNumber)
                    group by 1
                    order by numPurchasers desc;""", conn)
df.head()

,productName,numPurchasers
0,1992 Ferrari 360 Spider red,40
1,1934 Ford V8 Coupe,27
2,1952 Alpine Renault 1300,27
3,1972 Alfa Romeo GTA,27
4,Boeing X-32A JSF,27


## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [14]:
#Your code here
df = pd.read_sql_query("""select distinct employeeNumber,
                    officeCode,
                    o.city,
                    firstName,
                    lastName
                    from employees e
                    join offices o
                    using(officeCode)
                    join customers c
                    on e.employeeNumber = c.salesRepEmployeeNumber
                    join orders
                    using (customerNumber)
                    join orderdetails
                    using (orderNumber)
                    where productCode in (select productCode
                                                 from products
                                                 join orderdetails
                                                 using (productCode)
                                                 join orders
                                                 using (orderNumber)
                                                 group by 1
                                                 having count(distinct customerNumber) <= 20
                                         );""", conn)
df.head()

,employeeNumber,officeCode,city,firstName,lastName
0,1370,4,Paris,Gerard,Hernandez
1,1401,4,Paris,Pamela,Castillo
2,1621,5,Tokyo,Mami,Nishi
3,1501,7,London,Larry,Bott
4,1337,4,Paris,Loui,Bondur


## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [15]:
#Your code here
df = pd.read_sql_query("""select employeeNumber,
                            firstName,
                            lastName,
                            count(distinct customerNumber)
                            from employees e
                            join customers c
                            on e.employeeNumber = c.salesRepEmployeeNumber
                            group by 1,2,3
                            having AVG(creditLimit) > 15000;""", conn)
df.head()

,employeeNumber,firstName,lastName,count(distinct customerNumber)
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!